In [1]:
import postmit as pm
import xarray as xr

In [2]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=4, threads_per_worker=1)
client = Client(cluster) #dask_distributed.Client(cluster)
client

2022-04-20 12:12:25,005 - distributed.diskutils - INFO - Found stale lock file and directory '/home/jan/src/postmit/examples/dask-worker-space/worker-si81yuh8', purging
2022-04-20 12:12:25,006 - distributed.diskutils - INFO - Found stale lock file and directory '/home/jan/src/postmit/examples/dask-worker-space/worker-5rd6368h', purging
2022-04-20 12:12:25,006 - distributed.diskutils - INFO - Found stale lock file and directory '/home/jan/src/postmit/examples/dask-worker-space/worker-ing3ldt7', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 15.29 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44067,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 15.29 GiB
Comm: tcp://127.0.0.1:33789,Total threads: 1
Dashboard: http://127.0.0.1:34311/status,Memory: 3.82 GiB
Nanny: tcp://127.0.0.1:35299,


### Define paths
First we define the paths to the data and grid files, as well as the prefixes that we want to load. The package can handle `1y`, `1m`, `5d`, `1d` output that is assumed to be stored in the following format: `prefix_1y.iter.data`, etc. where iter is the iteration number.  
We can define a range of time steps to convert with the variables `start_niter`, `end_niter` and `step_niter`. `dt` is the model time step as required by `xmitgcm` to load the data.

In [3]:
path_to_data = "/home/jan/Data/MITgcm_test_data/"
path_to_grid = "/home/jan/Data/MITgcm_test_data/grid"

In [4]:
path_to_output = "/home/jan/Data/MITgcm_test_data/test_convert2nc/"
prefixes = ["2D_diags", "3D_diags"]
start_niter = 1555200
end_niter = 1557000
step_niter = 360
dt = 1200

#### Test loading of data
Below we test whether the loading of data works. `pm.convert.load_data` is just a wrapper for `xmitgcm.open_mds_dataset`. We open the files with different prefixes and merge them together.

In [5]:
data = xr.merge([pm.convert.load_data(path_to_data, path_to_grid, prefix, start_niter, end_niter, step_niter, dt, tout='5d') for prefix in ["2D_diags_5d", "3D_diags_5d"]])

In [6]:
data

<xarray.Dataset>
Dimensions:   (XC: 250, YC: 300, XG: 250, YG: 300, Z: 55, Zp1: 56, Zu: 55,
               Zl: 55, time: 6)
Coordinates: (12/34)
  * XC        (XC) >f4 5e+03 1.5e+04 2.5e+04 ... 2.475e+06 2.485e+06 2.495e+06
  * YC        (YC) >f4 5e+03 1.5e+04 2.5e+04 ... 2.975e+06 2.985e+06 2.995e+06
  * XG        (XG) >f4 0.0 1e+04 2e+04 3e+04 ... 2.47e+06 2.48e+06 2.49e+06
  * YG        (YG) >f4 0.0 1e+04 2e+04 3e+04 ... 2.97e+06 2.98e+06 2.99e+06
  * Z         (Z) >f4 -1.065 -3.345 -5.948 ... -3.728e+03 -3.877e+03 -4.027e+03
  * Zp1       (Zp1) >f4 0.0 -2.13 -4.56 ... -3.803e+03 -3.952e+03 -4.102e+03
    ...        ...
    dxF       (YC, XC) float32 dask.array<chunksize=(300, 250), meta=np.ndarray>
    dyF       (YC, XC) float32 dask.array<chunksize=(300, 250), meta=np.ndarray>
    dxV       (YG, XG) float32 dask.array<chunksize=(300, 250), meta=np.ndarray>
    dyU       (YG, XG) float32 dask.array<chunksize=(300, 250), meta=np.ndarray>
    iter      (time) int64 dask.array<chunksize=(6,), meta=np.ndarray>
  * time      (time) object 0060-12-28 12:00:00 ... 0061-01-23 12:00:00
Data variables: (12/28)
    MXLDEPTH  (time, YC, XC) float32 dask.array<chunksize=(6, 300, 250), meta=np.ndarray>
    ETAN      (time, YC, XC) float32 dask.array<chunksize=(6, 300, 250), meta=np.ndarray>
    oceQnet   (time, YC, XC) float32 dask.array<chunksize=(6, 300, 250), meta=np.ndarray>
    oceQsw    (time, YC, XC) float32 dask.array<chunksize=(6, 300, 250), meta=np.ndarray>
    oceFreez  (time, YC, XC) float32 dask.array<chunksize=(6, 300, 250), meta=np.ndarray>
    PHI_SURF  (time, YC, XC) float32 dask.array<chunksize=(6, 300, 250), meta=np.ndarray>
    ...        ...
    RHOAnoma  (time, Z, YC, XC) float32 dask.array<chunksize=(6, 55, 300, 250), meta=np.ndarray>
    VVEL      (time, Z, YG, XC) float32 dask.array<chunksize=(6, 55, 300, 250), meta=np.ndarray>
    UVEL      (time, Z, YC, XG) float32 dask.array<chunksize=(6, 55, 300, 250), meta=np.ndarray>
    WVEL      (time, Zl, YC, XC) float32 dask.array<chunksize=(6, 55, 300, 250), meta=np.ndarray>
    momHDiv   (time, Z, YC, XC) float32 dask.array<chunksize=(6, 55, 300, 250), meta=np.ndarray>
    PHIHYD    (time, Z, YC, XC) float32 dask.array<chunksize=(6, 55, 300, 250), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    title:        netCDF wrapper of MITgcm MDS binary data
    source:       MITgcm
    history:      Created by calling `open_mdsdataset(grid_dir='/home/jan/Dat...

### Conversion to [netcdf](https://www.unidata.ucar.edu/software/netcdf/)

In [7]:
pm.convert.convert2nc(path_to_data, path_to_grid, path_to_output, ds_out="test_convert2nc", prefixes=prefixes,
                      start=start_niter, end=end_niter, step=step_niter, dt=dt)

    -- loading 5d-output from binary
    -- running checks on 5d-data
    -- writing 5d-output to netCDF


'data saved to /home/jan/Data/MITgcm_test_data/test_convert2nc/'

In [8]:
!ls /home/jan/Data/MITgcm_test_data/test_convert2nc/

test_convert2nc_5d_0060.nc  test_convert2nc_5d_0061.nc


### Conversion to [zarr](https://zarr.readthedocs.io/en/stable/)
We define the same variables as above but this time we store the data to a [zarr store](https://zarr.readthedocs.io/en/stable/)

In [9]:
path_to_output = "/home/jan/Data/MITgcm_test_data/test_convert2zarr/"
prefixes = ["2D_diags", "3D_diags"]
start_niter = 1555200
end_niter = 1557000
step_niter = 360
dt = 1200

In [10]:
pm.convert.convert2zarr(path_to_data, path_to_grid, path_to_output, ds_out="test_convert2zarr", prefixes=prefixes,
                        start=start_niter, end=end_niter, step=step_niter, dt=dt)

    -- loading 5d-output from binary
    -- running checks on 5d-data
    -- writing 5d-output to zarr


'data saved to /home/jan/Data/MITgcm_test_data/test_convert2zarr/'

In [11]:
!ls /home/jan/Data/MITgcm_test_data/test_convert2zarr/

test_convert2zarr.5d.zarr
